# Lab 2

## Regressão Linear Múltipla

Vamos comparar os resultados de 3 regressões simples estimadas com os dados do banco *Advertisement*, uma para cada preditor e de uma regressão múltipla com os 3 preditores sendo controlados ao mesmo tempo.

In [0]:
import pandas as pd
import statsmodels.formula.api as sms
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
# Lendo os dados da web
adv = pd.DataFrame.from_csv("http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv")
adv.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,TV,radio,newspaper,sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


### 3 Regressões Simples

In [4]:
model_tv = sms.ols(formula="sales ~ TV", data=adv).fit()
print(model_tv.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     312.1
Date:                Wed, 27 Feb 2019   Prob (F-statistic):           1.47e-42
Time:                        19:08:47   Log-Likelihood:                -519.05
No. Observations:                 200   AIC:                             1042.
Df Residuals:                     198   BIC:                             1049.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.0326      0.458     15.360      0.0

In [5]:
model_radio = sms.ols(formula="sales ~ radio", data=adv).fit()
print(model_radio.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     98.42
Date:                Wed, 27 Feb 2019   Prob (F-statistic):           4.35e-19
Time:                        19:09:19   Log-Likelihood:                -573.34
No. Observations:                 200   AIC:                             1151.
Df Residuals:                     198   BIC:                             1157.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.3116      0.563     16.542      0.0

In [6]:
model_news = sms.ols(formula="sales ~ newspaper", data=adv).fit()
print(model_news.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     10.89
Date:                Wed, 27 Feb 2019   Prob (F-statistic):            0.00115
Time:                        19:09:35   Log-Likelihood:                -608.34
No. Observations:                 200   AIC:                             1221.
Df Residuals:                     198   BIC:                             1227.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.3514      0.621     19.876      0.0

### Adicionando os 3 preditores

In [8]:
# Vamos exibir os 3 parâmetros juntos só para facilitar a comparação
print("Modelo TV:\t" + str(model_tv.params[1]))
print("Modelo rádio:\t" + str(model_radio.params[1]))
print("Modelo News:\t" + str(model_news.params[1]))

Modelo TV:	0.047536640433019764
Modelo rádio:	0.20249578339243982
Modelo News:	0.05469309847227315


In [9]:
# Agora, estimando com os 3 preditores ao mesmo tempo
model_complete = sms.ols(formula="sales ~ TV + radio + newspaper", data=adv).fit()
print(model_complete.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Wed, 27 Feb 2019   Prob (F-statistic):           1.58e-96
Time:                        19:13:28   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9389      0.312      9.422      0.0

Perceba que o efeito da variável `newspaper` **desaparece**.  Isso acontece porque quando estimamos o efeito de newspaper controlando pelas outras variáveis, percebemos que o efeito de newspaper na verdade era relacionado à variável rádio devido ao fato das duas possuírem correlação. O efeito de newspaper estimado na regressão simples era, portanto, **efeito espúrio**. 

Vamos verificar a matriz de correlação dessas variáveis:

In [11]:
adv[["TV", "radio", "newspaper"]].corr()

,TV,radio,newspaper
TV,1.000000,0.054809,0.056648
radio,0.054809,1.000000,0.354104
newspaper,0.056648,0.354104,1.000000


Perceba que, enquanto os pares TV-Radio e TV-Newspaper tem uma correlação baixa, o par Radio-Newpaper tem uma correlação razoável (0,35).

Vamos também observar os intervalos de confiança dos estimadores:

In [26]:
model_complete.conf_int()

,0,1
Intercept,2.323762,3.554016
TV,0.043014,0.048516
radio,0.171547,0.205513
newspaper,-0.012616,0.010541


Agora, vamos usar nosso modelo para fazer predições.

In [22]:
yhat = model_complete.get_prediction()
print("Predictions:")
print(yhat.predicted_mean)

Predictions:
[20.52397441 12.33785482 12.30767078 17.59782951 13.18867186 12.47834763
 11.72975995 12.12295317  3.72734086 12.55084872  7.0322992  17.28512918
 10.57712073  8.82630048 18.43436638 20.81929952 12.82365674 23.22495716
  9.95168206 14.16607293 18.10076728 14.7405382   6.4891503  16.5459329
  8.14651887 15.6100386  14.98951429 17.05167344 19.41053803  9.14402389
 21.6339338  11.3460929   7.63888314 18.86426829  7.57483051 17.00682618
 23.40590052 15.62347779  9.90868103 20.44761039 16.37766467 17.2959832
 21.59580326 13.96385684  8.88787996 15.16152314  8.87338673 21.7226299
 16.26362018  8.1681656  12.63121132  9.33981296 20.66297563 19.94469957
 20.37443008 21.2926106   8.52771254 12.77458802 21.89805198 18.13348698
  5.74215558 22.89067208 16.78426073 13.21069202 16.97773556  7.84904532
  9.01603163 12.0370073  18.97657924 21.10891244 17.77949782 10.62693815
 10.36684881  9.90298206 17.32931197 11.85832174  4.47758904 13.81190223
  8.81331353  9.67530328 11.44592364 14.6

In [24]:
print("Confidence Intervals:")
print(yhat.conf_int())

Confidence Intervals:
[[19.99626571 21.05168311]
 [11.87464876 12.80106088]
 [11.64932004 12.96602152]
 [17.16942909 18.02622994]
 [12.67900667 13.69833705]
 [11.75402892 13.20266635]
 [11.33038649 12.12913341]
 [11.80438956 12.44151677]
 [ 3.14478088  4.30990084]
 [12.11556574 12.98613171]
 [ 6.61204334  7.45255506]
 [16.84732814 17.72293022]
 [ 9.99721407 11.15702739]
 [ 8.42453877  9.22806219]
 [18.09689396 18.77183881]
 [20.34009554 21.29850349]
 [11.8469578  13.80035569]
 [22.69494485 23.75496946]
 [ 9.61171822 10.2916459 ]
 [13.89410276 14.4380431 ]
 [17.71327214 18.48826242]
 [14.28398237 15.19709403]
 [ 5.96231839  7.01598222]
 [16.20198977 16.88987602]
 [ 7.77416659  8.51887115]
 [15.09348267 16.12659454]
 [14.63737507 15.34165351]
 [16.68013939 17.42320748]
 [19.02273501 19.79834104]
 [ 8.77103008  9.51701771]
 [21.15545795 22.11240965]
 [11.04742991 11.6447559 ]
 [ 7.18590745  8.09185883]
 [18.33118254 19.39735404]
 [ 7.11885321  8.03080782]
 [16.42231843 17.59133393]
 [22.7

# Exercício

### 1

Abra o banco de dados `Boston` e estime 3 regressões:

- A primeira prevendo `medv` usando como preditor `lstat`;
- A segunda prevendo `medv` usando como preditor `age`;
- A terceira usando ambos os preditores.

O que podemos dizer sobre o comportamento dos coeficientes estimados? E sobre o ajuste dos modelos?

### 2

Estime uma regressão usando todas as variáveis como preditoras. Todas são úteis para prever `medv`? Quais você deixaria e quais tiraria? Por quê?

Como foi o ajuste desse modelo?

Agora tire as variáveis que você considerou que não ajudam a estimação de `medv`. O que aconteceu com o ajuste do modelo?

In [27]:
# Lendo os dados
boston = pd.DataFrame.from_csv("https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv")
boston.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
crim,,,,,,,,,,,,,
0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
